In [10]:
p = 97
x_data = list(range(p))
y_data = list(range(1, p))

In [24]:
45 % 97

45

In [41]:
def get_label(x_data, y_data, x_idx, y_idx, p, operation):
    x, y = x_data[x_idx], y_data[y_idx]
    return operation(x, y, p) 

def div_mod(x, y, p):
    res = (x/y) % p
    return res


In [42]:
def test_div_mod():
    expected_results = [
        (0, 5, 0),
        (5, 5, 1),
        (90, 3, 30),
        (90, 2, 45)
    ]
    for x, y, result in expected_results:
        assert div_mod(x, y, 97) == result

test_div_mod()

In [91]:
def create_data(x_data, y_data, p, operation):
    """
    Returns: 
        `data` flattened 1D array of size len(x_data) * len(y_data)
    """
    data = {"operand": [], "label": []}
    for i in range(len(x_data)):
        for j in range(len(y_data)):
            data["operand"].append((x_data[i], y_data[j]))
            data["label"].append(get_label(x_data, y_data, i, j, p, operation))
    return data

data = create_data(x_data, y_data, p, div_mod)

In [92]:
def test_create_data():
    data = create_data(x_data, y_data, p, div_mod)
    assert len(data["operand"]) == len(data["label"]) == len(x_data) * len(y_data)

    expected_results = [(-1, (96,96), 1),
                        (0, (0, 1), 0),
                        (96, (1, 1), 1),
                        (192, (2, 1), 2)]

    for position, operand, label in expected_results:
        assert data["operand"][position] == operand
        assert data["label"][position] == label

test_create_data()